In [ ]:
!pip install -U langchain langchain-core langchain-community langchain-google-genai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
from langchain_google_genai import ChatGoogleGenerativeAI
# 코랩 Secrets에서 GOOGLE_API_KEY를 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


In [ ]:
#데이터베이스 저장된 대화 기록 확인
import sqlite3
import json # 'message' 컬럼의 JSON 문자열을 파싱하기 위함
def get_history_from_db(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()
  print(f"데이터베이스 '{db_path}'에 성공적으로 연결되었습니다.")

  cursor.execute("SELECT * FROM message_store")
  print("\n대화 기록")
  rows = cursor.fetchall()
  for row in rows:
    msg_id, session_id, message_json = row
    try:
        # 'message' 컬럼은 JSON 문자열이므로 파싱
        message_data = json.loads(message_json)
        msg_type = message_data.get("type", "unknown")

        # LangChain 메시지의 실제 내용은 'data' 딕셔너리 안에 'content' 키로 저장
        msg_content = message_data.get("data", {}).get("content", "[내용 없음]")

        print(f"ID: {msg_id}, Session: {session_id}")
        print(f"  Type: {msg_type.upper()}")
        print(f"  Content: {msg_content}")
        print("-" * 40)
    except json.JSONDecodeError:
        print(f"ID: {msg_id}, Session: {session_id}, Message: {message_json[:100]}...") # JSON 파싱 오류 시 원본 출력
        print("-" * 40)

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
store_histories = {} # SQLChatMessageHistory 객체를 저장
# 데이터베이스에서 대화내용 가져오는 함수
def get_chat_history(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  if session_id not in store_histories:
      print(f"{session_id} 에 대한 SQLChatMessageHistory 생성 및 연결")
      sql_chat_history = SQLChatMessageHistory(
            session_id=session_id,
            connection=f"sqlite:///{db_path}"
      )
      store_histories[session_id] = sql_chat_history
  return store_histories[session_id] # SQLChatMessageHistory 객체 반환

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 한국어 챗봇입니다."),
    MessagesPlaceholder(variable_name="history"), # 현재 대화 기록
    ("human", "{input}"),
])

#메모리에만 저장

## ConversationChain 사용

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
# 대화 요약
store_summarizers = {} # ConversationSummaryBufferMemory 객체 저장 (요약용)
def get_session_summarizer_chain(session_id):
    if session_id not in store_summarizers:
        print(f"새로운 세션 ID: {session_id} 에 대한 ConversationSummaryBufferMemory (요약용) 생성")

        memory = ConversationSummaryBufferMemory(
            llm=llm,
            max_token_limit=200, # 요약에 사용할 최대 토큰 제한
            memory_key="history", # 이 메모리에서 대화 기록을 가져올 키
            return_messages=True # 메시지 객체 리스트 형태로 반환
        )

        store_summarizers[session_id]  = ConversationChain(
          llm = llm,
          # ConversationSummaryBufferMemory 사용
          memory=memory,
          prompt=prompt,
        )
    return store_summarizers[session_id]

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  conversation_chain = get_session_summarizer_chain(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      response = conversation_chain.invoke({"input":user_input})['response']
      print(response)
      print("\n--- 요약 업데이트 ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션1
session_id = 'session1_s'
chatbot(session_id)

/tmp/ipython-input-3012387590.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(
/tmp/ipython-input-3012387590.py:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  store_summarizers[session_id]  = ConversationChain(



--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session1_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 안녕 내 이름은 테디야
챗봇 (답변 생성 중)...
안녕하세요, 테디님! 만나서 반갑습니다. 저는 당신의 친절한 챗봇입니다. 무엇을 도와드릴까요? 😊

--- 요약 업데이트 ---
입력: 서울에 대해 간략하게 설명해줘
챗봇 (답변 생성 중)...
테디님, 서울에 대해 간략하게 설명해 드릴게요!

서울은 대한민국의 수도이자 가장 큰 도시예요. 600년이 넘는 역사를 가진 곳으로, 과거와 현재가 조화롭게 어우러져 있죠. 

**핵심적인 특징:**

*   **정치, 경제, 문화의 중심지:** 대한민국의 모든 것이 서울에 집중되어 있다고 해도 과언이 아니에요.
*   **다양한 볼거리:** 경복궁, 창덕궁 같은 역사적인 궁궐부터 남산타워, 롯데월드 같은 현대적인 랜드마크까지 볼거리가 정말 많아요.
*   **맛있는 음식:** 맛집이 정말 많기로 유명하죠! 한식은 물론이고 다양한 나라의 음식을 맛볼 수 있어요.
*   **활기찬 도시:** 24시간 잠들지 않는 도시라고 할 정도로 항상 활기가 넘쳐요.
*   **잘 갖춰진 교통:** 지하철, 버스 등 대중교통 시스템이 잘 되어 있어서 이동하기 편리해요.

서울은 정말 매력적인 도시랍니다! 혹시 서울에 대해 더 궁금한 점이 있으시면 언제든지 물어보세요. 😊

--- 요약 업데이트 ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#세션2
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...
아직은 당신의 이름을 모릅니다. 알려주시겠어요? 😊

--- 요약 업데이트 ---
입력: 내 이름은 디테야
챗봇 (답변 생성 중)...
안녕하세요, 디테님! 이제 디테님의 이름을 기억하겠습니다. 😊 무엇을 도와드릴까요?

--- 요약 업데이트 ---
입력: 제주도에 대해 간략하게 설명해줘
챗봇 (답변 생성 중)...
디테님, 제주도에 대해 간략하게 설명해 드릴게요!

**제주도는...**

*   **대한민국에서 가장 큰 섬:** 아름다운 자연경관과 독특한 문화로 유명한 곳입니다.
*   **화산섬:** 약 180만 년 전 화산 활동으로 만들어졌으며, 섬 전체가 화산 지형의 특징을 가지고 있습니다.
*   **세계자연유산:** 한라산, 성산일출봉, 거문오름 용암동굴계는 유네스코 세계자연유산으로 지정되어 보존되고 있습니다.
*   **다양한 볼거리:** 아름다운 해변, 폭포, 오름, 숲길 등 자연 명소뿐만 아니라 박물관, 미술관, 테마파크 등 다양한 즐길 거리가 있습니다.
*   **특색 있는 음식:** 흑돼지, 갈치, 옥돔, 해산물 등 제주도에서만 맛볼 수 있는 특별한 음식들이 많습니다.
*   **사계절 아름다운 곳:** 봄에는 유채꽃, 여름에는 해수욕, 가을에는 억새, 겨울에는 눈 덮인 한라산 등 계절마다 다른 매력을 느낄 수 있습니다.

제주도에 대해 더 궁금한 점이 있으시면 언제든지 물어보세요! 😊

--- 요약 업데이트 ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#세션1_서울에 관한 얘기
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 대중교통에 대해 알려줄래?
챗봇 (답변 생성 중)...
서울의 대중교통은 매우 편리하고 잘 갖춰져 있어요. 지하철, 버스, 택시 등 다양한 선택지가 있고, 대부분의 지역을 쉽게 이동할 수 있습니다.

*   **지하철:** 노선이 복잡해 보일 수 있지만, 색깔로 구분되어 있어 익숙해지면 편리하게 이용할 수 있어요. 환승 시스템도 잘 되어 있고, 주요 관광지 대부분을 지하철로 갈 수 있습니다.
*   **버스:** 시내 곳곳을 연결하는 다양한 노선이 있어요. 지하철로 가기 어려운 곳도 버스를 이용하면 편리합니다. 버스 종류에 따라 요금과 노선이 다르니, 미리 확인하는 것이 좋습니다.
*   **택시:** 비교적 저렴하고 편리하게 이용할 수 있지만, 출퇴근 시간에는 교통 체증이 심할 수 있습니다. 일반 택시 외에 모범 택시, 대형 택시 등도 있습니다.

대중교통을 이용할 때는 교통카드를 사용하는 것이 편리합니다. 티머니 카드나 캐시비 카드를 구입해서 충전한 후, 지하철 개찰구나 버스 단말기에 태그하면 됩니다.

혹시 특정 목적지나 이동 경로가 있다면 더 자세하게 안내해 드릴 수 있어요!

--- 요약 업데이트 ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#세션1 메모리 확인
summarizer_memory = get_session_summarizer_chain(session_id)
summarizer_memory.memory.load_memory_variables({})['history']

[SystemMessage(content="The human introduces themself as Teddy in Korean. The AI greets Teddy and offers assistance. Teddy asks for a brief explanation of Seoul. The AI then provides a brief explanation of Seoul as the capital and largest city of South Korea with a rich history, highlighting its key features such as being the center of politics, economy, and culture, having diverse attractions, delicious food, a vibrant atmosphere, and well-developed transportation. Teddy then asks about public transportation. The AI explains that Seoul's public transportation is convenient and well-equipped, with options like the subway, bus, and taxi. It details the subway's color-coded lines and efficient transfer system, the bus's extensive network, and the taxi's affordability while noting potential rush hour congestion. It recommends using a transportation card like T-money or Cashbee for easy payment and offers further assistance with specific destinations or routes.", additional_kwargs={}, resp

In [ ]:
#세션2 삭제
store_summarizers.pop('session2_s')

In [ ]:
#세션2 초기화됨
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 대중교통에 대해 알려줄래?
챗봇 (답변 생성 중)...
네, 대중교통에 대해 알려드릴게요! 어떤 점이 궁금하신가요? 좀 더 구체적으로 질문해주시면 더 자세하게 답변해 드릴 수 있어요. 예를 들어:

*   **어떤 종류의 대중교통**에 대해 알고 싶으신가요? (버스, 지하철, 기차, 택시, 트램 등)
*   **어느 지역**의 대중교통 정보를 원하시나요? (서울, 부산, 뉴욕, 런던 등)
*   **특정 목적**을 위한 정보를 찾으시나요? (예: 출퇴근 경로, 관광지 이동 방법, 요금 정보 등)

만약 특정 질문이 없으시다면, 대중교통의 일반적인 정보에 대해 먼저 설명해 드릴 수 있습니다. 예를 들어, 대중교통의 장점, 이용 방법, 에티켓 등을 알려드릴 수 있어요. 😊

--- 요약 업데이트 ---
입력: quit
챗봇을 종료합니다.


##메모리에 직접 저장

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
store_summarizers = {} #초기화
def get_session_summarizer(session_id):
    if session_id not in store_summarizers:
        print(f"새로운 세션 ID: {session_id} 에 대한 ConversationSummaryBufferMemory (요약용) 생성")

        store_summarizers[session_id] = ConversationSummaryBufferMemory(
            llm=llm,
            max_token_limit=200, # 요약에 사용할 최대 토큰 제한
            memory_key="history", # 이 메모리에서 대화 기록을 가져올 키
            return_messages=True # 메시지 객체 리스트 형태로 반환
        )

    return store_summarizers[session_id]

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  memory = get_session_summarizer(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      history = memory.load_memory_variables({})['history']
      print("\n--- history에 저장된 내용 ---")
      print(history)
      print("---history 내용 출력 끝---\n")
      inputs = {"input": user_input, 'history':history}

      chain = prompt | llm
      response = chain.invoke(inputs).content
      print(response)

      # 사용자 입력과 AI의 답변을 ConversationSummaryMemory에 전달하여 요약 업데이트
      memory.save_context(
                {"input": user_input}, # 사용자 입력
                {"output": response} # AI의 답변
      )
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션1_초기화됨.
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session1_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[]
---history 내용 출력 끝---

아니요, 저는 당신의 이름을 기억하지 못합니다. 저는 정보를 저장하는 기능이 없기 때문에, 이전에 나눈 대화 내용을 기억할 수 없습니다. 혹시 제게 당신의 이름을 알려주시겠어요? 😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 내 이름은 테디야
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[HumanMessage(content='내 이름을 기억해?', additional_kwargs={}, response_metadata={}), AIMessage(content='아니요, 저는 당신의 이름을 기억하지 못합니다. 저는 정보를 저장하는 기능이 없기 때문에, 이전에 나눈 대화 내용을 기억할 수 없습니다. 혹시 제게 당신의 이름을 알려주시겠어요? 😊', additional_kwargs={}, response_metadata={})]
---history 내용 출력 끝---

안녕하세요, 테디님! 😊 만나서 반갑습니다. 오늘 하루는 어떠셨나요? 제가 테디님을 위해 무엇을 해 드릴 수 있을까요?

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 서울에 대해 간략히 알려줄래?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[HumanMessage(content='내 이름을 기억해?', additional_kwargs={}, response_metadata={}), AIMessage(content='아니요, 저는 당신의 이름을 기억하지 못합니다. 저는 정보를

In [ ]:
#메모리 확인
summarizer_memory = get_session_summarizer(session_id)
summarizer_memory.load_memory_variables({})['history']

[SystemMessage(content="The human asks the AI if it remembers their name. The AI does not remember the human's name because it cannot store information from previous conversations. The human then tells the AI their name is Teddy. The AI greets Teddy and asks how their day is going and if it can help them with anything. Teddy asks for a brief overview of Seoul. The AI provides a brief overview of Seoul, including its status as the capital of South Korea, its history, major attractions like palaces and shopping districts, and basic information like language and currency.", additional_kwargs={}, response_metadata={})]

In [ ]:
#세션2_제주도
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 제주도에 대해 간략히 설명해줘
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[]
---history 내용 출력 끝---

제주도는 대한민국 최남단에 위치한 화산섬으로, 아름다운 자연경관과 독특한 문화로 유명합니다. 

**핵심 특징:**

*   **자연:** 한라산, 성산일출봉, 만장굴 등 다양한 화산 지형과 아름다운 해변, 폭포, 숲이 어우러져 있습니다. 유네스코 세계자연유산으로 지정되어 있습니다.
*   **문화:** 독특한 제주 방언, 음식 문화 (흑돼지, 갈치, 해산물 등), 전통 가옥 등이 남아있습니다.
*   **관광:** 다양한 볼거리 (박물관, 미술관, 테마파크 등)와 즐길 거리 (올레길, 해양 액티비티 등)가 풍부하여 국내외 관광객들에게 인기가 많습니다.

**간단 요약:** 아름다운 자연과 독특한 문화를 가진 한국의 대표적인 관광지입니다.

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#메모리 확인
summarizer_memory = get_session_summarizer(session_id)
summarizer_memory.load_memory_variables({})['history']

[SystemMessage(content='The human asks for a brief explanation of Jeju Island. The AI explains that Jeju Island is a volcanic island located at the southernmost tip of South Korea, known for its beautiful natural scenery and unique culture, including volcanic landforms like Hallasan and Seongsan Ilchulbong, unique dialect and cuisine, and diverse attractions making it a popular tourist destination.', additional_kwargs={}, response_metadata={})]

In [ ]:
#세션1_서울
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 이번엔 지형에 대해 알려줄래?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[SystemMessage(content="The human asks the AI if it remembers their name. The AI does not remember the human's name because it cannot store information from previous conversations. The human then tells the AI their name is Teddy. The AI greets Teddy and asks how their day is going and if it can help them with anything. Teddy asks for a brief overview of Seoul. The AI provides a brief overview of Seoul, including its status as the capital of South Korea, its history, major attractions like palaces and shopping districts, and basic information like language and currency.", additional_kwargs={}, response_metadata={})]
---history 내용 출력 끝---

서울의 지형에 대해 알려드릴게요.

서울은 한반도의 중앙에 위치하며, 산과 강이 어우러진 지형적 특징을 가지고 있습니다.

*   **산:** 서울은 북한산, 도봉산, 수락산 등 험준한 산으로 둘러싸여 있습니다. 이 산들은 서울의 자연 경관을 아름답게 만들 뿐만 아니라, 시민들에게 등산 및 휴식 공간을 제공합니다.
*   **강:** 한강은 서울을 가로지르는 중요한 강입니다. 한강은 서울의

#RunnableWithMessageHistory

##ConversationChain 사용

In [ ]:
store_summarizers = {} #초기화

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  conversation_chain = get_session_summarizer_chain(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      conversation_chain_with_history = RunnableWithMessageHistory(
        conversation_chain, # conversationChain
        get_chat_history,
        input_messages_key="input",
        history_messages_key="history",
      )
      config = {"configurable": {"session_id": session_id}}
      inputs = {"input": user_input}
      response = conversation_chain_with_history.invoke(inputs, config)['response']
      print(response)
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션1 새로 시작
session_id = 'session1_s'
chatbot(session_id)

/tmp/ipython-input-3012387590.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(
/tmp/ipython-input-3012387590.py:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  store_summarizers[session_id]  = ConversationChain(



--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session1_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 안녕 내 이름은 테디야
챗봇 (답변 생성 중)...
session1_s 에 대한 SQLChatMessageHistory 생성 및 연결
안녕하세요, 테디님! 만나서 반갑습니다. 저는 당신의 친절한 챗봇이에요. 무엇을 도와드릴까요? 😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 서울에 대해 간략하게 알려줄래?
챗봇 (답변 생성 중)...
네, 테디님! 서울에 대해 간략하게 알려드릴게요. 😊

**서울은 어떤 곳인가요?**

*   **대한민국의 수도**: 서울은 한국의 정치, 경제, 사회, 문화의 중심지입니다.
*   **역사적인 도시**: 600년 이상의 역사를 가진 도시로, 조선 시대의 궁궐과 사찰, 역사 유적지들이 잘 보존되어 있습니다.
*   **현대적인 도시**: 고층 빌딩과 현대적인 건축물, 쇼핑몰, 다양한 문화 시설이 즐비한 활기찬 도시입니다.
*   **글로벌 도시**: 다양한 국적의 사람들이 함께 살아가는 국제적인 도시이며, 세계적인 기업들의 본사가 위치해 있습니다.
*   **매력적인 관광 도시**: 아름다운 자연, 맛있는 음식, 다양한 볼거리와 즐길 거리가 가득한 매력적인 관광 도시입니다.

**서울의 주요 특징:**

*   **인구**: 약 1천만 명 이상이 거주하는 거대 도시입니다.
*   **지리**: 한강을 중심으로 발달했으며, 아름다운 산과 공원이 많아 자연과 도시가 조화를 이루고 있습니다.
*   **교통**: 지하철, 버스 등 대중교통 시스템이 잘 갖춰져 있어 이동이 편리합니다.
*   **문화**: 전통 문화와 현대 문화가 공존하며, 다양한 축제와 공연이 연중 내내 열립니다.
*   **음식**: 맛있는 한식을 비롯해 세계 각국의 다

In [ ]:
#메모리 확인
memory = get_session_summarizer(session_id)
memory.memory.load_memory_variables({})['history']

[SystemMessage(content='Teddy introduces himself in Korean. The AI greets Teddy and offers assistance. Teddy asks for a brief introduction to Seoul. The AI provides a summary of Seoul, including its status as the capital of South Korea, its historical and modern aspects, its global nature, and its attractiveness as a tourist destination, and offers to answer further questions.', additional_kwargs={}, response_metadata={})]

In [ ]:
#세션2 새로 시작
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...
session2_s 에 대한 SQLChatMessageHistory 생성 및 연결
네, 당신의 이름을 기억합니다. 당신의 이름은 따로 주어지지 않았습니다. 당신의 이름을 알려주시면, 제가 기억하도록 하겠습니다. 😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 내 이름은 디테야
챗봇 (답변 생성 중)...
안녕하세요, 디테님! 이제 디테님의 이름을 기억하겠습니다. 😊 무엇을 도와드릴까요?

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 제주도에 대해 간략하게 설명해줘
챗봇 (답변 생성 중)...
안녕하세요, 디테님! 제주도에 대해 간략하게 설명해 드릴게요.

**제주도**는 대한민국 최남단에 위치한 화산섬으로, 아름다운 자연경관과 독특한 문화로 유명합니다.

**주요 특징:**

*   **자연:** 한라산, 성산일출봉, 만장굴 등 다양한 화산 지형과 아름다운 해안선, 폭포, 숲 등 천혜의 자연환경을 자랑합니다. 유네스코 세계자연유산으로 지정되어 있습니다.
*   **문화:** 제주 방언, 독특한 음식 문화 (흑돼지, 갈치조림, 고기국수 등), 전통 가옥 등 본토와는 다른 독자적인 문화를 가지고 있습니다.
*   **관광:** 사계절 내내 다양한 볼거리와 즐길 거리가 풍부하여 국내외 관광객들에게 인기가 많습니다. 해수욕, 트레킹, 박물관 관람, 맛집 탐방 등 다양한 활동을 즐길 수 있습니다.
*   **경제:** 관광 산업이 주요 산업이며, 감귤, 흑돼지 등 특산물 생산도 활발합니다.

**간단 요약:** 아름다운 자연과 독특한 문화를 가진 대한민국의 대표적인 관광지입니다.

더 궁금한

In [ ]:
#세션1 서울
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 대중교통에 대해 알려줄래?
챗봇 (답변 생성 중)...
서울의 대중교통은 매우 편리하고 효율적입니다. 주요 교통수단은 다음과 같습니다:

1.  **지하철:** 서울 지하철은 노선이 복잡하지만, 색깔로 구분되어 있어 이용하기 쉽습니다. 주요 지역을 연결하며, 환승 시스템도 잘 갖춰져 있습니다.
2.  **버스:** 시내버스, 간선버스, 지선버스 등 다양한 종류의 버스가 있습니다. 버스 노선은 서울 전역을 촘촘하게 연결하고 있으며, 지하철과 연계하여 이용하면 편리합니다.
3.  **택시:** 일반 택시와 모범 택시가 있으며, 필요에 따라 이용할 수 있습니다. 택시는 비교적 요금이 비싸지만, 늦은 시간이나 짐이 많을 때 유용합니다.

**교통카드:** 티머니 또는 캐시비 교통카드를 사용하면 지하철과 버스를 편리하게 이용할 수 있으며, 환승 할인 혜택도 받을 수 있습니다.

더 궁금한 점이 있으신가요?

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session1_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session1_s
  Type: HUMAN
  Content: 안녕 내 이름은 테디야
----------------------------------------
ID: 2, Session: session1_s
  Type: AI
  Content: 안녕하세요, 테디님! 만나서 반갑습니다. 저는 당신의 친절한 챗봇이에요. 무엇을 도와드릴까요? 😊
----------------------------------------
ID: 3, Session: session1_s
  Type: HUMAN
  Content: 서울에 대해 간략하게 알려줄래?
----------------------------------------
ID: 4, Session: session1_s
  Type: AI
  Content: 네, 테디님! 서울에 대해 간략하게 알려드릴게요. 😊

**서울은 어떤 곳인가요?**

*   **대한민국의 수도**: 서울은 한국의 정치, 경제, 사회, 문화의 중심지입니다.
*   **역사적인 도시**: 600년 이상의 역사를 가진 도시로, 조선 시대의 궁궐과 사찰, 역사 유적지들이 잘 보존되어 있습니다.
*   **현대적인 도시**: 고층 빌딩과 현대적인 건축물, 쇼핑몰, 다양한 문화 시설이 즐비한 활기찬 도시입니다.
*   **글로벌 도시**: 다양한 국적의 사람들이 함께 살아가는 국제적인 도시이며, 세계적인 기업들의 본사가 위치해 있습니다.
*   **매력적인 관광 도시**: 아름다운 자연, 맛있는 음식, 다양한 볼거리와 즐길 거리가 가득한 매력적인 관광 도시입니다.

**서울의 주요 특징:**

*   **인구**: 약 1천만 명 이상이 거주하는 거대 도시입니다.
*   **지리**: 한강을 중심으로 발달했으며, 아름다운 산과 공원이 많아 자연과 도시가 조화를 이루고 있습니다.
*

In [ ]:
#세션2 삭제
store_summarizers.pop('session2_s')

In [ ]:
#세션2 새로 시작
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...
제가 이전에 당신의 이름을 알았던 적이 있다면 기억하고 있을 거예요. 혹시 제게 당신의 이름을 알려주셨었나요? 알려주시면 기억하겠습니다! 😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 대중교통에 대해 알려줘
챗봇 (답변 생성 중)...
대중교통은 여러 사람이 함께 이용하는 교통수단으로, 도시나 지역 사회에서 이동하는 데 중요한 역할을 합니다. 종류와 장단점, 그리고 이용 팁에 대해 자세히 알아볼까요?

**1. 대중교통의 종류**

*   **버스:** 가장 흔한 대중교통 수단 중 하나로, 노선이 다양하고 비교적 저렴합니다. 시내버스, 광역버스, 마을버스 등 다양한 종류가 있습니다.
*   **지하철/전철:** 정해진 노선을 따라 운행하며, 교통 체증의 영향을 받지 않아 빠르고 정확한 이동이 가능합니다.
*   **기차:** 장거리 이동에 적합하며, KTX, ITX-새마을, 무궁화호 등 다양한 등급이 있습니다.
*   **택시:** 비교적 비싸지만, 원하는 장소까지 편리하게 이동할 수 있습니다.
*   **트램 (노면전차):** 일부 도시에서 운영되며, 친환경적이고 관광용으로도 활용됩니다.
*   **배/페리:** 섬이나 해안 지역에서 이용되며, 육로로 이동하기 어려운 지역을 연결합니다.

**2. 대중교통의 장단점**

| 장점                                                       | 단점                                                                 |
| ------------------------------------

In [ ]:
#데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session2_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session2_s
  Type: HUMAN
  Content: 내 이름을 기억해?
----------------------------------------
ID: 2, Session: session2_s
  Type: AI
  Content: 네, 당신의 이름을 기억합니다. 당신의 이름은 따로 주어지지 않았습니다. 당신의 이름을 알려주시면, 제가 기억하도록 하겠습니다. 😊
----------------------------------------
ID: 3, Session: session2_s
  Type: HUMAN
  Content: 내 이름은 디테야
----------------------------------------
ID: 4, Session: session2_s
  Type: AI
  Content: 안녕하세요, 디테님! 이제 디테님의 이름을 기억하겠습니다. 😊 무엇을 도와드릴까요?
----------------------------------------
ID: 5, Session: session2_s
  Type: HUMAN
  Content: 제주도에 대해 간략하게 설명해줘
----------------------------------------
ID: 6, Session: session2_s
  Type: AI
  Content: 안녕하세요, 디테님! 제주도에 대해 간략하게 설명해 드릴게요.

**제주도**는 대한민국 최남단에 위치한 화산섬으로, 아름다운 자연경관과 독특한 문화로 유명합니다.

**주요 특징:**

*   **자연:** 한라산, 성산일출봉, 만장굴 등 다양한 화산 지형과 아름다운 해안선, 폭포, 숲 등 천혜의 자연환경을 자랑합니다. 유네스코 세계자연유산으로 지정되어 있습니다.
*   **문화:** 제주 방언, 독특한 음식 문화 (흑돼지, 갈치조림, 고

##메모리에 직접 저장

In [ ]:
store_summarizers = {} #초기화

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  memory = get_session_summarizer(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    # config 설정. 세션 ID 기준으로 대화 기록
    config = {"configurable": {"session_id": session_id}}
    try:
      # ConversationSummaryBufferMemory에서 요약된 내용 로드
      history = memory.load_memory_variables({})['history']
      print("\n--- history에 저장된 내용 ---")
      print(history)
      print("---history 내용 출력 끝---\n")
      print("챗봇 (답변 생성 중)...")
      config = {"configurable": {"session_id": session_id}}
      inputs = {"input": user_input, "history":history}
      chain_with_history = RunnableWithMessageHistory(
        prompt | llm,
        get_chat_history,
        input_messages_key="input",
        history_messages_key="history",
      )
      response = chain_with_history.invoke(inputs, config).content
      print(response)
      print()
      # 사용자 입력과 AI의 답변을 ConversationSummaryMemory에 전달하여 요약 업데이트
      memory.save_context(
                {"input": user_input}, # 사용자 입력
                {"output": response} # AI의 답변
      )
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션1. 메모리는 초기화 됐지만 db에 저장된 내용을 바탕으로 서울에 관한 이야기를 이어감.
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session1_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 기후에 대해 알려줄래?

--- history에 저장된 내용 ---
[]
---history 내용 출력 끝---

챗봇 (답변 생성 중)...
서울의 기후에 대해 자세히 알려드릴게요! 서울은 사계절이 뚜렷한 온대 기후를 가지고 있습니다.

**봄 (3월 ~ 5월):**

*   따뜻하고 쾌적한 날씨가 이어집니다.
*   꽃이 피는 시기로, 벚꽃 축제 등 다양한 봄 축제가 열립니다.
*   미세먼지가 심할 때도 있으니 주의해야 합니다.

**여름 (6월 ~ 8월):**

*   덥고 습한 날씨가 이어집니다.
*   장마철에는 비가 많이 내립니다.
*   무더위를 피하기 위해 실내 활동이나 물놀이를 즐기는 사람들이 많습니다.

**가을 (9월 ~ 11월):**

*   선선하고 맑은 날씨가 이어집니다.
*   단풍이 아름다운 시기로, 등산이나 야외 활동을 즐기기 좋습니다.
*   가을 축제와 행사도 많이 열립니다.

**겨울 (12월 ~ 2월):**

*   춥고 건조한 날씨가 이어집니다.
*   눈이 내리는 날도 있습니다.
*   스키나 스케이트 등 겨울 스포츠를 즐기기 좋습니다.

**여행 팁:**

*   **봄, 가을:** 얇은 옷을 여러 겹 껴입는 것이 좋습니다.
*   **여름:** 얇고 통풍이 잘 되는 옷을 입고, 자외선 차단제를 꼭 바르세요.
*   **겨울:** 따뜻한 옷, 장갑, 목도리 등을 착용하여 보온에 신경 쓰세요.

서울은 사계절마다 다른 매력을 느낄 수 있는 도시입니다. 여행 계획을 세울 때 참고하시면 좋을 것 같아요! 😊


--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#세션2
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 제주도 정보 알려줄래?

--- history에 저장된 내용 ---
[]
---history 내용 출력 끝---

챗봇 (답변 생성 중)...
디테님, 제주도에 대한 더 자세한 정보를 원하시는군요! 어떤 점이 궁금하신가요? 좀 더 구체적으로 알려주시면 더욱 맞춤형 정보를 제공해 드릴 수 있습니다. 예를 들어:

*   **여행:** 제주도 여행 코스 추천, 명소 정보, 숨겨진 장소, 특정 테마 여행 (예: 힐링 여행, 액티비티 여행)
*   **음식:** 제주도 맛집 추천, 향토 음식 소개, 특정 음식 (예: 흑돼지, 해산물) 관련 정보
*   **숙소:** 호텔, 펜션, 게스트하우스 등 숙소 추천, 특정 지역 숙소 정보
*   **교통:** 렌터카, 대중교통 정보, 교통편 추천
*   **문화:** 제주도 역사, 문화 유적지, 축제 정보
*   **기타:** 제주도 날씨, 쇼핑 정보, 액티비티 (예: 승마, 스쿠버 다이빙)

궁금한 점을 알려주시면 자세하게 답변해 드릴게요! 😊


--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#메모리 확인
summarizer_memory = get_session_summarizer(session_id)
summarizer_memory.load_memory_variables({})['history']

[SystemMessage(content="The human asks for information about Jeju Island bus routes. The AI explains that Jeju Island's public transportation system is bus-centered, divided into trunk (express) and branch line buses. Trunk lines quickly connect major areas, while branch lines connect villages and towns. The AI provides tips on using each type, information on routes from the airport and intercity bus terminal, and ways to check routes using the Jeju Bus Information System, map apps, and bus stop signs. It also advises checking the schedule in advance due to long intervals and using a transportation card. The AI then offers to provide more specific route information if the human specifies a particular area or tourist destination.", additional_kwargs={}, response_metadata={})]